<a href="https://colab.research.google.com/github/zilingggg/-artificial-intelligence/blob/main/s1022139__%3CHW1%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Wed Mar 20 04:16:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!ln -s /content/drive/MyDrive/ /my_drive
!ls
!ls /my_drive

Mounted at /content/drive
drive  sample_data
 B048D719-C405-4FB9-8555-2C54446970E0.jpeg  'My Drive'		 Untitled1.ipynb    未命名文件.gdoc
'Colab Notebooks'			     MyDrive		 yolov4-tiny_1103   照片
 GoodNotes				    'python 爬網.gdoc'	 yolov4-tiny_1123   爬網.ipynb
 mp3					     Untitled0.ipynb	 yolov4-tiny_1214


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/food-11'


testing  training  validation


In [ ]:
import os
import time
import cv2
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torch.optim import Adam
import torchvision.transforms as transforms
from torch.autograd import Variable

def readfile(path, label):

    image_dir = sorted(os.listdir((path)))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)

    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):

        img = cv2.imread(os.path.join(path, file))
        x[i, :, :, :] = cv2.resize(img, (128,128))
        if label:
            y[i] = int(file.split("_")[0])

    if label:
        return x, y
    else:
        return x

workspace_dir = '/content/drive/MyDrive/Colab Notebooks/food-11'
print("Reading data--------->")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print(f'Size of validation data = {len(val_x)}')
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of testing data = {}".format(len(test_x)))


Reading data--------->
Size of training data = 9866
Size of validation data = 3430
Size of testing data = 3347


In [ ]:

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x

        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform

    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform)
val_set = ImgDataset(val_x, val_y, test_transform)
train_loader = DataLoader(train_set , batch_size= batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
print("Data loading and preprocessing completed.")



Data loading and preprocessing completed.


In [ ]:

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
        )

        self.fc = nn.Sequential(
            nn.Linear(512* 4* 4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,11)
        )
    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)


In [ ]:
model = Classifier()


if torch.cuda.is_available():
    model = model.cuda()

loss = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss = loss.cuda()


optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

num_epoch = 45

for epoch in range(num_epoch):
    epoch_start_time = time.time()

    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train()
    for data in train_loader:
        img,targets = data
        if torch.cuda.is_available():
            img = img.cuda()
            targets = targets.cuda()

        optimizer.zero_grad()

        train_pred = model(img)

        batch_loss = loss(train_pred,targets)

        batch_loss.backward()

        optimizer.step()

        train_acc += (train_pred.argmax(1)==targets).sum()
        train_loss += batch_loss.item()

    model.eval()
    with torch.no_grad():
        for data in val_loader:
            img, targets = data
            if torch.cuda.is_available():
                img = img.cuda()
                targets = targets.cuda()
            val_pred = model(img)
            batch_loss = loss(val_pred,targets)

            val_acc += (val_pred.argmax(1) == targets).sum()
            val_loss += batch_loss.item()
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % (epoch + 1, num_epoch, time.time() - epoch_start_time, train_acc / train_set.__len__(), train_loss / train_set.__len__(), val_acc / val_set.__len__(),val_loss / val_set.__len__()))



[001/045] 25.11 sec(s) Train Acc: 0.244577 Loss: 0.017500 | Val Acc: 0.244315 loss: 0.016401
[002/045] 25.19 sec(s) Train Acc: 0.346645 Loss: 0.014673 | Val Acc: 0.266472 loss: 0.018195
[003/045] 25.05 sec(s) Train Acc: 0.403000 Loss: 0.013433 | Val Acc: 0.377551 loss: 0.014125
[004/045] 25.49 sec(s) Train Acc: 0.453578 Loss: 0.012396 | Val Acc: 0.376385 loss: 0.016624
[005/045] 25.11 sec(s) Train Acc: 0.480438 Loss: 0.011764 | Val Acc: 0.500875 loss: 0.011719
[006/045] 25.28 sec(s) Train Acc: 0.514798 Loss: 0.010997 | Val Acc: 0.466764 loss: 0.012319
[007/045] 24.97 sec(s) Train Acc: 0.545814 Loss: 0.010303 | Val Acc: 0.505539 loss: 0.011503
[008/045] 25.07 sec(s) Train Acc: 0.563957 Loss: 0.009942 | Val Acc: 0.503790 loss: 0.011850
[009/045] 24.99 sec(s) Train Acc: 0.597203 Loss: 0.009289 | Val Acc: 0.446939 loss: 0.015579
[010/045] 25.08 sec(s) Train Acc: 0.622643 Loss: 0.008612 | Val Acc: 0.567055 loss: 0.010164
[011/045] 25.00 sec(s) Train Acc: 0.643929 Loss: 0.008011 | Val Acc: 0

In [ ]:
train_val_x = np.concatenate((train_x,val_x),axis=0)
train_val_y = np.concatenate((train_y,val_y),axis=0)

train_val_set = ImgDataset(train_val_x,train_val_y,train_transform)
train_val_lodaer = DataLoader(train_val_set,batch_size=batch_size,shuffle=True)


model_best = Classifier()
if torch.cuda.is_available():
    model_best = model_best.cuda()

loss = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss = loss.cuda()

optimizer = torch.optim.Adam(model_best.parameters(),lr=0.001)

num_epoch = 45
for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    for data in train_val_lodaer:
        img,targets = data
        if torch.cuda.is_available():
            img = img.cuda()
            targets = targets.cuda()

        optimizer.zero_grad()
        train_pred = model_best(img)
        batch_loss = loss(train_pred,targets)
        batch_loss.backward()
        optimizer.step()

        train_acc += (train_pred.argmax(1) == targets).sum()
        train_loss += batch_loss.item()
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % (epoch + 1, num_epoch, time.time()-epoch_start_time, train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))



[001/045] 29.98 sec(s) Train Acc: 0.281588 Loss: 0.016370
[002/045] 29.92 sec(s) Train Acc: 0.385304 Loss: 0.013756
[003/045] 29.77 sec(s) Train Acc: 0.441712 Loss: 0.012572
[004/045] 30.05 sec(s) Train Acc: 0.485259 Loss: 0.011577
[005/045] 29.91 sec(s) Train Acc: 0.535349 Loss: 0.010466
[006/045] 30.08 sec(s) Train Acc: 0.573782 Loss: 0.009671
[007/045] 29.90 sec(s) Train Acc: 0.596646 Loss: 0.009017
[008/045] 30.23 sec(s) Train Acc: 0.626730 Loss: 0.008362
[009/045] 30.06 sec(s) Train Acc: 0.654257 Loss: 0.007741
[010/045] 29.71 sec(s) Train Acc: 0.669073 Loss: 0.007385
[011/045] 29.73 sec(s) Train Acc: 0.686673 Loss: 0.007007
[012/045] 30.02 sec(s) Train Acc: 0.711567 Loss: 0.006451
[013/045] 29.83 sec(s) Train Acc: 0.728640 Loss: 0.006141
[014/045] 29.84 sec(s) Train Acc: 0.744058 Loss: 0.005718
[015/045] 30.03 sec(s) Train Acc: 0.764064 Loss: 0.005364
[016/045] 29.90 sec(s) Train Acc: 0.767223 Loss: 0.005172
[017/045] 30.17 sec(s) Train Acc: 0.778354 Loss: 0.004958
[018/045] 29.9

In [ ]:
# 將模型移動到GPU上
model_best = model_best.cuda()

# 將輸入數據轉換為PyTorch張量，並將其移動到GPU上
test_set = ImgDataset(torch.tensor(test_x).cuda(), transform=test_transform


In [ ]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)
#將結果寫入 csv 檔
with open("predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))
